# Statistical Timeseries Methods - One example at a time

Let's look at candidate methods for forecasting a single timeseries. These will be:

- Holt-Winters (Exponential Smoothing)
- SARIMAX
- Prophet

This notebook will look at one ISBN country example at at time (to allow me to focus!)

The four candiates are:

- ISBN 9788490369968 (a Kid's Box title) in Spain (ES)
- ISBN 9781316628744 (a Kid's Box title) in Turkey (TR) - more peaky
- ISBN 9781108457651 (an English Grammar in Use title) in South Korea (KR) - very sparse
- ISBN 9780521174909 (a reader) in Spain (ES) - no clear pattern


In [ ]:
#Import libraries
from helpers import *

import psycopg2
import numpy as np
import pandas as pd
import datetime as dt
import dateutil

from statsmodels.tsa.holtwinters import ExponentialSmoothing as hwes
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

from prophet import Prophet

import matplotlib.pyplot as plt

In [ ]:
#Redshift user credentials - set here
USER = 
PASSWORD = 

FCST_PERIOD = 12   #How many months I want to forecast ahead - let's start witg full 12 months

In [ ]:
#Create SQLAlchemy engine for Redshift database
user = USER
password = PASSWORD
host= 
port='5439'
dbname='prod'

url = "postgresql+psycopg2://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, dbname)
engine = create_engine(url)

## 1. Get demand data from Amazon Redshift

Read the demand data into a dataframe using a key list of the 3 ISBN/country combinations above

In [ ]:
key_list = ['9788490369968ES',
            '9781316628744TR',
            '9781108457651KR',
            '9780521174909ES']

key = '9788490369968ES'

In [ ]:
query = f"""
    select 
        isbn + ship_to_country_key as key,
        isbn,
        ship_to_country_key as country,
        last_day(date) as month,
        sum(quantity_demanded) as qty
    from r2ibp.f_demand_actual
    where key = '{key}'
    and month <= current_date
    group by month, isbn, ship_to_country_key
    order by month asc
    """

conn = engine.connect()
df_demand = pd.read_sql_query(query, conn)
conn.close()

In [ ]:
ts = convert_to_ts(df_demand, start = '2018-08-31') # so always plot full date range
        
plt.plot(ts)
plt.grid()
plt.title(key)      
plt.show();

## 2. Holt-Winters

Let's start with hyperparameters already selected i.e. what I've used in later examples.

I'll add a step on hyperparameter selection later

### Forecast using Holt-Winters

In [ ]:
#Helper functions

def predict_ts_using_hwes(df_demand, period, config = ['add', False, 'add', False]):
    
    t,d,s,b = config       
      
    ts_actuals = convert_to_ts(df_demand, period)
    #Add a tiny value to avoid divide by zero errors
    ts_actuals['qty'] += 1e-10   

    ts_train = ts_actuals[:-period]
    ts_test = ts_actuals[-period:]
    ts_naive1 = ts_actuals[-(12+period):-12].shift(periods = 12, freq = 'M')

    mod = hwes(ts_train,  
            trend= t,
            damped_trend = d,
            seasonal= s,
            use_boxcox = b,
            seasonal_periods = 12,
            initialization_method="estimated",
            freq = 'M'
            )
    res = mod.fit()
    pred = res.predict(len(ts_train),len(ts_train)+(period-1))
            
    #If a negatve forecast, when using additive set to zero?
    #220221 - play with this as should make HW worse
    #pred[pred<0] = 0

    df_forecast = ts_test.copy().reset_index()
    df_forecast.insert(loc=0, column='key', value=key)
    df_forecast.rename(columns = {'index':'month', 'qty':'actuals'}, inplace = True)
    df_forecast['pred'] = pred.values
    df_forecast['naive-1'] = ts_naive1['qty'].values
   
    return df_forecast


def calc_rsme_values(df_forecast):
    
    y_test = df_forecast['actuals'] #Rename this when I tidy up!
    yhat = df_forecast['pred']
    y_naive1 = df_forecast['naive-1']       

    rmse_pred = round(mean_squared_error(y_test, yhat, squared = False), 2)
    rmse_naive1 = round(mean_squared_error(y_test, y_naive1, squared = False), 2)
    
    return rmse_pred, rmse_naive1


def plot_preds_against_actuals(df_demand, df_forecast, period):
   
    key = df_demand['key'][0]
    
    ts_actuals = convert_to_ts(df_demand, period)
    ts_naive1 = ts_actuals[-(12+period):-12].shift(periods = 12, freq = 'M')

    #Reconstruct the HW forecast
    ts_pred = df_forecast.copy()[['month', 'pred']]
    ts_pred.set_index(pd.to_datetime(ts_pred.month), inplace=True)
    ts_pred.drop("month", axis=1, inplace=True)

    plt.plot(ts_actuals[-24:], '-o', label="actuals") #Only plot 2 years back
    plt.plot(ts_pred, '-o', label="pred")
    plt.plot(ts_naive1, '-o', label="naive-1")
    plt.grid()
    plt.legend(fontsize=12)
    plt.title(key);

    plt.show();

### Grid Search on config to find best HWES parameters

In [ ]:
#NB This has to be done with training data only so need to lop FCST_PERIOD off df_demand

current_date = dt.datetime.today()
dt_fcst_period_ago = dt.date(current_date.year, current_date.month, 1)\
                            - dateutil.relativedelta.relativedelta(months=FCST_PERIOD)

df_demand_search = df_demand.copy()
df_demand_search = df_demand_search[df_demand_search['month'] < dt_fcst_period_ago]

In [ ]:
%%capture --no-display
#Grid search likely to create lots of warnings
#https://stackoverflow.com/questions/40105796/turn-warning-off-in-a-cell-jupyter-notebook

configs = list()

t_params = ['add', 'mul']   #trend
d_params = [True, False]    #damped
s_params = ['add', 'mul']   #seasonal
b_params = [True, False]    #use_boxcox

# create config instances
for t in t_params:
    for d in d_params:
        for s in s_params:
            for b in b_params:
                cfg = [t,d,s,b] 
                configs.append(cfg)

#Now score each config using RMSE
scores = []

for config in configs:
    try:
        df_hwes_forecast = predict_ts_using_hwes(df_demand_search, FCST_PERIOD, config)       
        rmse_pred, _ = calc_rsme_values(df_hwes_forecast)
    except:
        rmse_pred = None
     
    scores.append([config, rmse_pred])
    
df_scores = pd.DataFrame(scores, columns = ['trend, damped, seasonal, BoxCox', 'rmse']).sort_values(by='rmse')

In [ ]:
df_scores

#This will vary by case and I'll need at least 12 + 2 forecast periods to do this every time

In [ ]:
#Let's look at what the best fit looks like for this combo
config = df_scores.iloc[0,0]

df_hwes_forecast = predict_ts_using_hwes(df_demand, FCST_PERIOD, config)
plot_preds_against_actuals(df_demand, df_hwes_forecast, FCST_PERIOD)

rmse_pred, rmse_naive1 = calc_rsme_values(df_hwes_forecast)
print('RMSE pred =', rmse_pred)
print('RMSE naive-1  =', rmse_naive1)
print('Pred better than naive-1?', rmse_pred < rmse_naive1)

### Compare with results for pre-selected hyperparameters

In [ ]:
CONFIG = ['add', False, 'add', False]

df_hwes_forecast = predict_ts_using_hwes(df_demand, FCST_PERIOD, config = CONFIG)

plot_preds_against_actuals(df_demand, df_hwes_forecast, FCST_PERIOD)

rmse_pred, rmse_naive1 = calc_rsme_values(df_hwes_forecast)
print('RMSE pred =', rmse_pred)
print('RMSE naive-1  =', rmse_naive1)
print('Pred better than naive-1?', rmse_pred < rmse_naive1)

## 3. SARIMAX

Same as above and even more important to put in the hyperparameter selection

### Forecast with SARIMAX

In [ ]:
#This is running a simple additative trend and additative seasonality model

def predict_ts_using_SARIMAX(df_demand, period, order=(0,1,1), seasonal_order=(1,1,0,12)):
    
    #This is a repeat of HWES above
    ts_actuals = convert_to_ts(df_demand, period)
    ts_train = ts_actuals[:-period]
    ts_test = ts_actuals[-period:]
    ts_naive1 = ts_actuals[-(12+period):-12].shift(periods = 12, freq = 'M')
    
    mod = SARIMAX(
        ts_train,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False)

    res = mod.fit()
    pred = res.predict(len(ts_train),len(ts_train)+(period-1))
    
    #Append the results to df_forecasts
    df_forecast = ts_test.copy().reset_index()
    df_forecast.insert(loc=0, column='key', value=key)
    df_forecast.rename(columns = {'index':'month', 'qty':'actuals'}, inplace = True)
    df_forecast['pred'] = pred.values
    df_forecast['naive-1'] = ts_naive1['qty'].values
    
    return df_forecast

### Tuning SARIMAX.

What is the best selection of order and seasonal order parameters?

### Start by looking at the timeseries

Things to look at are:

- Timeseries Decomposition
- Autocorrelation
- Differencing

In [ ]:
ts_actuals = convert_to_ts(df_demand)

In [ ]:
#This is the timeseries decomposition
#See M7/05 lesson for an example of this

from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.seasonal import STL #Need to remind myself why used this

res = seasonal_decompose(ts_actuals, two_sided = False)

trend = res.trend
seasonal = res.seasonal
residual = res.resid

# Plot the results
#plt.subplots(figsize = (15,5))

plt.plot(ts_actuals, label = 'Original')
plt.plot(trend, label = 'Trend')
plt.plot(seasonal, label = 'Seasonality')
plt.plot(residual, label = 'Residuals')
plt.title(key)
plt.grid()
plt.legend()
plt.show();


In [ ]:
#This is the autocorrelation
#Uses pd.Series.autocorr

lags = range(1, 25)
autocorrs = [ts_actuals['qty'].autocorr(lag=lag) for lag in lags]
plt.stem(lags, autocorrs, use_line_collection=True)
plt.xlabel("Lag", fontsize=12)
plt.ylabel("Autocorrelation", fontsize=12);
plt.show();


In [ ]:
#This is the differencing
#Which looks broadly stationary

plt.plot(ts_actuals['qty'].diff(1))
plt.grid()
plt.show();

In [ ]:
#This is the differenced version showing the yearly autocorrelation
#NB There is also a PSCF version of this

lags = range(1, 25)
autocorrs = [ts_actuals['qty'].diff(1).autocorr(lag=lag) for lag in lags]
plt.stem(lags, autocorrs, use_line_collection=True)
plt.xlabel("Lag", fontsize=12)
plt.ylabel("Autocorrelation", fontsize=12);
plt.show();

### SARIMAX (p,d,q)(P,D,Q,s) parameter search

Let's brute force a solution here.
Assume that only need to difference once and only ARMA a few steps 

In [ ]:
import warnings
import itertools
warnings.filterwarnings("ignore") # specify to ignore warning messages

# Set p, and q to take values from 0 to 3. 
p = q = range(0, 4)
# Set d to take 1
d = range(1,2)

pdq = list(itertools.product(p, d, q))
PDQs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
#This didn't take long despite 16*16 permutations

ts_train = ts_actuals[:-FCST_PERIOD]

mse_values = []
order_values = []

for i in pdq:
    for j in PDQs:
    # Generate a SARIMA model with the selected parameters for the seasonal order
        mod = SARIMAX(ts_train,
                    order=i,
                    seasonal_order=j,
                    enforce_stationarity=False,
                    enforce_invertibility=False,
                    freq = 'M')
    
        results = mod.fit()
        mse = results.mse
       
        mse_values.append(mse)
        order_values.append([i,j])

In [ ]:
min_index = mse_values.index(min(mse_values))
best_params = order_values[min_index]

print('Best training mse:', mse_values[min_index])
print('Best params:', order_values[min_index])

### Run Best Parameters

In [ ]:
df_SARIMAX_forecast = predict_ts_using_SARIMAX(df_demand,
                                               FCST_PERIOD, order=best_params[0], seasonal_order=best_params[1])

plot_preds_against_actuals(df_demand, df_SARIMAX_forecast, FCST_PERIOD)

rmse_pred, rmse_naive1 = calc_rsme_values(df_SARIMAX_forecast)
print('RMSE pred =', rmse_pred)
print('RMSE naive-1  =', rmse_naive1)
print('Pred better than naive-1?', rmse_pred < rmse_naive1)

#This is actually worse than setting logically (see below)

### Check the residuals

See L7 AI M7/06 i.e. that the difference between our prediction and actuals are uncorrelated and have zero mean.
In fact there are other tests for checking any regression model

In [ ]:
df_residuals = df_SARIMAX_forecast.copy()
df_residuals['qty'] = df_residuals['actuals'] - df_residuals['pred']

print('Mean of residuals is', round(df_residuals['qty'].mean(),1))

ts_residuals = df_residuals.copy()[['month', 'qty']]
ts_residuals.set_index(pd.to_datetime(ts_residuals.month), inplace=True)
ts_residuals.drop("month", axis=1, inplace=True)

plt.plot(ts_residuals)
plt.grid()
plt.title('Residuals')      
plt.show();

### Compare with results for pre-selected hyperparameters

In [ ]:
#This is a default set of parameters that is equivalenet to same as last year (PDQs = 1,1,0,12)
#And adjust based on the error term from last month (pdq = 0,1,1)

ORDER=(0,1,1)
SEASONAL_ORDER=(1,1,0,12)

df_SARIMAX_forecast = predict_ts_using_SARIMAX(df_demand, FCST_PERIOD, order=ORDER, seasonal_order=SEASONAL_ORDER)

plot_preds_against_actuals(df_demand, df_SARIMAX_forecast, FCST_PERIOD)

rmse_pred, rmse_naive1 = calc_rsme_values(df_SARIMAX_forecast)
print('RMSE pred =', rmse_pred)
print('RMSE naive-1  =', rmse_naive1)
print('Pred better than naive-1?', rmse_pred < rmse_naive1)

## 4. Prophet

NB This works in a different way to the other two. It requires a dataframe formatted in a particular way

See L7 AI M7/07
The use of linear trend is justified because of the previous analyses for SARIMAX re the underlying trend

In [ ]:
def predict_ts_using_Prophet(df_demand, period):
    
    ts_actuals = df_demand.copy()[['month', 'qty']]
    ts_actuals = ts_actuals.rename(columns={'month': 'ds', 'qty': 'y'}) 
    ts_train = ts_actuals[:-FCST_PERIOD]
    ts_test = ts_actuals[-FCST_PERIOD:]
    ts_naive1 = ts_actuals.copy()[-(12+FCST_PERIOD):-12]
    ts_naive1['ds'] = ts_test['ds'].values
    
    mod = Prophet(
        growth="linear",
        daily_seasonality=False,
        weekly_seasonality=False,
        yearly_seasonality=True)

    res = mod.fit(ts_train)
    pred = res.predict(ts_test)
    
    df_forecast = ts_test.copy()
    df_forecast.insert(loc=0, column='key', value=key)
    df_forecast.rename(columns = {'ds':'month', 'y':'actuals'}, inplace = True)
    df_forecast['pred'] = pred['yhat'].to_list()
    df_forecast['naive-1'] = ts_naive1['y'].to_list()
    
    return df_forecast
    
    

In [ ]:
df_Prophet_forecast = predict_ts_using_Prophet(df_demand, FCST_PERIOD)

plot_preds_against_actuals(df_demand, df_Prophet_forecast, FCST_PERIOD)

rmse_pred, rmse_naive1 = calc_rsme_values(df_Prophet_forecast)
print('RMSE pred =', rmse_pred)
print('RMSE naive-1  =', rmse_naive1)
print('Pred better than naive-1?', rmse_pred < rmse_naive1)